# Instalación con docker

docker pull docker.elastic.co/elasticsearch/elasticsearch:7.15.2

docker run -p 127.0.0.1:9200:9200 -p 127.0.0.1:9300:9300 -e "discovery.type=single-node" -v /Users/julian/Documents/repos/SQL/data/elasticsearch:/usr/share/elasticsearch/data docker.elastic.co/elasticsearch/elasticsearch:7.15.2



# !pip install elasticsearch-dsl

In [2]:
from elasticsearch_dsl import connections, Document, Document, Keyword, Integer, Index

In [18]:
es = connections.create_connection(
    alias="default",
    hosts=['http://127.0.0.1:9200'],
    timeout=1000,
    retry_on_timeout=True,
    max_retries=10,
    verify_certs=False
)

In [19]:
class User(Document):
    name = Keyword()
    address = Keyword()
    age = Integer()
    
    class Index:
        name = 'users'
        settings = {
            'number_of_shards': 1,
            'number_of_replicas': 0
        }

In [20]:
User.init()

In [21]:
# Creación de usuario
user1 = User(name='Pedro Martinez', address='Alsina 1234', age=45)
user2 = User(name='Marina Gomez', address='Libertad 4321', age=83)

In [22]:
# Guardo Usuario
user1.save()
user2.save()

'created'

In [23]:
User.search().count()

2

In [30]:
users = User.search().filter().execute()

In [31]:
users.timed_out

False

In [32]:
for u in users:
    print(u.to_dict(), u.meta.id)

{'name': 'Pedro Martinez', 'address': 'Alsina 1234', 'age': 45} 0e1ReH0B71PntoApWV8c
{'name': 'Marina Gomez', 'address': 'Libertad 4321', 'age': 83} 0u1ReH0B71PntoApWV8q
{'name': 'Maria Diaz', 'address': 'Guayra 2323', 'age': 21, 'gender': 'M'} 0-1SeH0B71PntoApFl-6


In [10]:
u.meta.to_dict()

{'index': 'users',
 'id': '0O1LeH0B71PntoApPl--',
 'score': 0.0,
 'doc_type': '_doc'}

In [24]:
Index('users').get_mapping()

{'users': {'mappings': {'properties': {'address': {'type': 'keyword'},
    'age': {'type': 'integer'},
    'name': {'type': 'keyword'}}}}}

In [25]:
# Si no lo tiene configurado lo infiere 'gender'
user3 = User(name='Maria Diaz', address='Guayra 2323', age=21, gender='M')

In [26]:
user3.save()

'created'

In [29]:
for u in users:
    print(u.to_dict(), u.meta.id)

{'name': 'Pedro Martinez', 'address': 'Alsina 1234', 'age': 45} z-1LeH0B71PntoApPl9L
{'name': 'Marina Gomez', 'address': 'Libertad 4321', 'age': 83} 0O1LeH0B71PntoApPl--


In [28]:
Index('users').get_mapping()

{'users': {'mappings': {'properties': {'address': {'type': 'keyword'},
    'age': {'type': 'integer'},
    'gender': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'name': {'type': 'keyword'}}}}}